In [5]:
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd

In [4]:
load_dotenv()

pg_user = os.getenv('PG_USER')
pg_password = os.getenv('PG_PASSWORD')
pg_host = os.getenv('PG_HOST')
pg_port = os.getenv('PG_PORT')
pg_db = os.getenv('PG_GPU_DB')

pg_conn_str = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}"
pg_engine = create_engine(pg_conn_str)

Descriptive Analytics

Business Question: What is the average GPU price for different levels of memory size and clock speed?

In [21]:
sql_query = '''
WITH joined_specs AS (
    SELECT
        api."productName",
        api."memSize",
        api."gpuClock",
        web."MSRP"
    FROM sql_project.gpu_specs_raw api
    JOIN sql_project.gpu_hierarchy_web_raw web
        ON api."productName" = web."Graphics Card"
    WHERE web."MSRP" IS NOT NULL
)
SELECT
    CAST(js."memSize" AS NUMERIC) AS mem_size_gb,
    js."gpuClock" AS gpu_clock_mhz,
    AVG(CAST(REGEXP_REPLACE(js."MSRP", '[$,]', '', 'g') AS NUMERIC)) AS avg_price_usd
FROM joined_specs js
GROUP BY mem_size_gb, gpu_clock_mhz
ORDER BY avg_price_usd DESC;
'''
df = pd.read_sql_query(sql_query, con=pg_engine)
print(df)

    mem_size_gb  gpu_clock_mhz  avg_price_usd
0          28.0            900         2000.0
1          24.0            975         1600.0
2          24.0            810         1000.0
3          12.0           1312          800.0
4          20.0           1065          750.0
5          16.0            900          750.0
6          12.0           1245          550.0
7          12.0           1825          550.0
8          12.0           1290          400.0
9           8.0            650          330.0
10         16.0           1825          330.0
11          8.0           1200          300.0
12          8.0           1140          270.0


In [ ]:
pd.set_option('display.max_rows', None)
display(df)

Diagnostic Analytics

Business Question: Which specs included in our GPUs can generate higher sale prices?

In [ ]:
sql_query = '''
WITH joined_specs AS (
    SELECT
        api."productName",
        api."memSize",
        api."gpuClock",
        web."MSRP",
        api."unifiedShader"
    FROM sql_project.gpu_specs_raw api
    JOIN sql_project.gpu_hierarchy_web_raw web
        ON api."productName" = web."Graphics Card"
    WHERE web."MSRP" IS NOT NULL
),
ranked_specs AS (
    SELECT *,
           AVG(CAST(REGEXP_REPLACE(js."MSRP", '[$,]', '', 'g') AS NUMERIC)) OVER() AS overall_avg_price,
           RANK() OVER (ORDER BY js."MSRP" DESC) AS price_rank
    FROM joined_specs js
)
SELECT
    rs."productName",
    rs."memSize",
    rs."gpuClock",
    rs."unifiedShader",
    rs."MSRP",
    rs.overall_avg_price,
    CAST(REGEXP_REPLACE(rs."MSRP", '[$,]', '', 'g') AS NUMERIC) - rs.overall_avg_price AS price_above_avg
FROM ranked_specs rs
WHERE CAST(REGEXP_REPLACE(rs."MSRP", '[$,]', '', 'g') AS NUMERIC) > rs.overall_avg_price
ORDER BY price_above_avg DESC
LIMIT 15;
'''
df = pd.read_sql_query(sql_query, con=pg_engine)
print(df)